                            ## SEQUENCE ANALYSIS DEMO ##
                            (INSIGHT TECHNICAL 7/22/2019)

### Basic problem:

You are studying a molecule is involved in disease. You would like to create a drug that disrupt's the molecule's function and cures the disease. To solve this problem, you first need to understand how the molecule works. **In turn, to understand how a molecule works, you need to find individual residues that contribute to function.**

### Classic wet-lab solution to this problem is slow:

1. Mutate the molecule by changing one of its residues
2. Isolate the mutated molecule
3. Subject the mutated molecule to a fitness test - is this one residue critical to function?
4. Repeat for every residue

One residue takes days to weeks, assuming it even works (mutant may not purify, there may be no functional assay, etc).

### The alternative solution is comparative analysis of related molecules:

1. Any given molecule has relatives (homologs) in other species.
2. Find residues that stay the same in each homolog (are conserved), versus those that change.
3. Residues that stay the same are probably the ones responsible for the fundamental function -- **TEST THEM FIRST!!!**


                            ## SEQUENCE ANALYSIS DEMO ##
                            (INSIGHT TECHNICAL 7/22/2019)

## Sample case study - RNAse P
1. Responsible for processing of tRNAs (molecules that shuffle amino acids to the ribosome)<br>
2. Involved in human disease (form of dwarfism)<br>

#### Goal: find functional sites in RNAse P using comparative sequence analysis

In [ ]:
addpath '../code_matlab/'

### Fetch the molecule of interest from Protein Data Bank

In [ ]:
structure = getpdb('3q1q');

In [ ]:
structure.Model(1).Atom(1)

### Display the structure

In [ ]:
%plot native
%molviewer(structure) %Jmol - built into MATLAB

% USE PYMOL

### Load/find/make alignment of RNAse P homologs
Option 1 - BLAST<br>
Option 2 - premade INFERNAL alignments on Rfam<br>

In [ ]:
%plot inline
% load alignment
aln_fp = 'input_data/RF00010.msf';
aln = multialignread(aln_fp);
aln

In [ ]:
%aln(1).Header
%aln(1).Sequence
%aln(2).Sequence

In [ ]:
seqalignviewer(aln)

### Clean the aligment
1. Remove overly long (duplicates) or short sequences (fragments)
2. Remove non-standard characters

In [ ]:
addpath '../code_matlab/'
seq_len = structure.Sequence(1).NumOfResidues;
[clean_aln, ~] = clean_alignment(aln, seq_len);
clean_aln

In [ ]:
seqalignviewer(clean_aln)

### Profile 3Q1Q query sequence to alignment

In [ ]:
atoms = struct2table(structure.Model(1).Atom);
chain = 'B';
format_flag = 'pdb';
[~,pdb_res,pdb_seq_prof,~] = profile_pdbcif_chain_to_aln(clean_aln,...
                                  atoms, chain, format_flag);
% add profiled sequence to alignment
nseq = length(clean_aln);
clean_aln(nseq+1).Sequence = pdb_seq_prof;
clean_aln(nseq+1).Header = 'pdb_query';

In [ ]:
clean_aln
seqalignviewer(clean_aln)

### Calculate conservation (information entropy) of each column in the aligment

\begin{equation*}
r_{i}= -\sum_{a=1}^{n}f_{a}ln(f_{a})
\end{equation*}

Column $i$ featuring 10 bases that are:<br>
entirely conserved ($f_{1} = 1$) will produce $r_{i}=0$<br>
has even split between 2 bases ($f_{1} = f_{2} = 0.5$) will produce $r_{i}=xyz$<br>
is split between 10 uniuqe bases ($f_{1},.,f_{n} =0.1$) will produce $r_{i}=xyz$<br>



In [ ]:
int_aln = format_sequence(clean_aln);
entropy_scores = calculate_entropy(int_aln);
result = [pdb_res, entropy_scores];

%convert entropy ranks to percentile/threshold ranks
coverage = [];
for i=1:length(entropy_scores)
    at_or_below_rank = sum(entropy_scores <= entropy_scores(i));
    coverage = [coverage; at_or_below_rank/length(entropy_scores)];
end
result = [result, coverage];
result_table = array2table(result, 'VariableNames',...
                           {'pdb_res', 'raw_rank', 'coverage'});

fp = 'data/rnasep_sample_ranks.csv';
csvwrite(fp,result);

### Map $r_{i}$ over linear sequence of RNAse P

Pattern 1: bases clearly have different distributions

In [ ]:
hold on;
plot(pdb_res, entropy_scores, 'black');
%plot(pdb_res, movmean(entropy_scores,10), 'red')
ylabel('entropy score')
xlabel('position in sequence')
hold off;

### Map $r_{i}$ over 3D structure

--show in pymol

Pattern 2: top-ranked bases cluster in 3D space

### Measure statistical significance of clustering

1. Convert 3D representation into an adjacency matrix
2. Calculate expected number of contact pairs vs rank threshold
3. Calculate observed number of contact pairs vs rank threshold
4. Convert to z-score

In [ ]:
% convert 3D atom structure into an adjacency matrix
[adj_matrix, seq_table] = pdbtable2distmatrix(atoms, chain); %4A

In [ ]:
%plot inline
image(adj_matrix, 'CDataMapping','scaled')
xlabel('residue i')
ylabel('residue j')

In [ ]:
%trim matrix to include only residues for which we have ranks
matrix_res = seq_table.pdb_resSeq;
keep = find(ismember(matrix_res, pdb_res));
trace_matrix = adj_matrix(keep,keep);

In [ ]:
% Calculate clustering z-score
% Calculate expected clustering table
% z-score code only needs half the matrix (off diagonal)
lower_trig = trace_matrix;
lower_trig = tril(trace_matrix, -1);
exp_table = calc_expected_clust(lower_trig);
% calculate clustering z-score
clust_z = calc_clust_z_score(lower_trig, result_table, exp_table);

In [ ]:
clust_z

In [ ]:
hold on;
plot(clust_z.cov_bin, clust_z.obs_clust_weight, 'red', 'LineWidth', 1);
plot(clust_z.cov_bin, clust_z.exp_clust_weight, 'black', 'LineWidth', 1);
title('Number of observed contact pairs exceeds random expected')
box on;
xlabel('fraction top residues selected (coverage)')
ylabel('contact pairs formed by residues')
legend('observed', 'expected', 'Location', 'NorthWest')
hold off;

In [ ]:
plot(clust_z.cov_bin, clust_z.z_score, 'black', 'LineWidth', 1);
title({'Number of observed contact pairs exceeds random expected\\n',...
       '(top-ranked residues cluster together)'})
xlabel('fraction top residues selected (coverage)')
ylabel('clustering z-score')

### Measure overlap of top residues with known functional sites
Pattern 3: clusters define functional sites

In [ ]:
% Calculate overlap z-scores
% access active sites (pdb labels of active residues)
fs_fp = 'input_data/3Q1Q_RF00010_functional_site_COMMENTS.txt';
fs = importdata(fs_fp);
fs = fs.data;
% find these residues in the trace residue list
trace_fs = find(ismember(pdb_res, fs));
% calculate overlap of top residues with known active sites
olap_z = calc_overlap_z_score(result_table.coverage, trace_fs);

In [ ]:
olap_z

In [ ]:
hold on;
plot(olap_z.cov_bin, olap_z.observed, 'red', 'LineWidth', 1);
plot(olap_z.cov_bin, olap_z.expected, 'black', 'LineWidth', 1);
title('Number of known functional residues in selection exceeds random expectate')
xlabel('fraction top residues selected (coverage)')
ylabel('number of functional residues in selection')
legend('observed', 'expected', 'Location', 'NorthWest')
hold off;

In [ ]:
plot(olap_z.cov_bin, olap_z.z_score, 'black', 'LineWidth', 1);
title('Number of known functional residues in selection exceeds random expectated');
xlabel('fraction top residues selected (coverage)');
ylabel('overlap z-score');
ylabel('clustering z-score');

In [ ]:
% calculate accuracy in form of ROC AUC
labels = zeros(height(result_table),1);
labels(1:10) = 1;
[x,y,~,auc] = perfcurve(labels, result_table.coverage, 1);
box on;
hold on;
plot(x, y, 'Color', 'red', 'LineWidth', 1);
plot([0 1],[0,1],'Color', 'black', 'LineWidth', 1);
legend('entropy ranks', 'random', 'Location', 'NorthWest');
title('Entropy ranks are predictive of known functional sites');
xlabel('FPR');
ylabel('TPR');
hold off;


In [ ]:
% export data
fp = 'data/rnasep_sample_ranks.csv';
writetable(olap_z, 'results/clust_rnase_p.csv');
writetable(clust_z, 'results/clust_rnase_p.csv');

### Summary, 3 patterns:

1. Residues can be ranked based on their evolutionary history, ranks are different
2. Top-ranked residues co-locolize and form clusters
3. Clusters overlap functional sites<br>
<br>
4. Did not show today: clustering is predictive of accuracy - tigheter clusters better predict the functional site. CAN OPTIMIZE ALIGNMENTS FOR BETTER PREDICTION.

### Actionable recommendation to wet-lab scientists:
1. If desiring to study existing sites, focus on the highest ranked residues.
2. If want to focus on novel sites, exclude known sites from analysis, and focus on the highest remaining residues.


### Optimize alignment

1. Generate ensemble of alignments
2. Measure clustering and accuracy for each alignment
3. Plot clustering vs accuracy <-- should be a strong postive correlation

In [ ]:
% code here